In [ ]:
---
date: "2025-05-06T15:00:00.00Z"
description: "A principled approach for evaluating LLM workflows"
published: true
tags:
  - python
  - llm
  - ragas
  - pydantic-ai
  - evals
time_to_read: 10
title: "📈 Evaluation Driven Development (EDD) with PydanticAI"
type: post
image: "/public/images/llm-vision/banner.png"
---


When embarking on any AI implementation, I prioritise establishing a robust evaluation framework. The initial development of this evaluation process itself provides valuable insights into effective assessment criteria. Here are the critical aspects I consider when selecting evaluation tools:

### Developer Experience
The evaluation environment must be optimized for rapid iteration and reliable feedback. An effective framework:

- Enables quick, dependable, and observable experiment execution
- Helps troubleshoot issues rather than creating additional friction
- Simplifies the addition of new test scenarios

### Composability
Identifying meaningful performance metrics requires deliberate consideration and experimentation. Optimal evaluation might be composed of several metrics including:

- Rules based metrics (e.g., BLEU score for NLP tasks)
- Reference-free assessments (like LLM-based judges)
- Human annotated labels

PydanticAI has recently released Evals which provides an interesting and simple approach which may tick a few of my boxes above. Let's try this out with a simple finantial markets text summarisation use-case. We'll be building up the evaluation suite as we go, following the principles of Evaluation Driven Development (EDD).

## Setup

In [ ]:
%%capture
!uv pip install --upgrade pydantic-evals 'pydantic-ai-slim[bedrock]' pydantic-graph boto3 logfire

In [1]:
%%capture
!logfire auth
!logfire projects use stephenhib-blog

import logfire
from pydantic_ai import Agent

logfire.configure(send_to_logfire='if-token-present', scrubbing=False)
Agent.instrument_all()

In [2]:
from pydantic_evals import Case, Dataset
from datasets import load_dataset

import nest_asyncio
nest_asyncio.apply()

def convert_hf_to_pydantic_dataset(
    hf_dataset_name,
    input_column="user_input",
    split="train",
    subset=None,
):
    """
    Convert a Hugging Face dataset to a PydanticAI Dataset
    
    Args:
        hf_dataset_name: Name of the Hugging Face dataset
        input_column: Column to use as Case inputs
        output_column: Column to use as model outputs (if available)
        split: Dataset split to use
    
    Returns:
        A Pydantic Dataset object
    """
    # Load the Hugging Face dataset
    hf_dataset = load_dataset(hf_dataset_name, split=split)
    
    # Convert each row to a Pydantic Case
    cases = []
    for i, item in enumerate(hf_dataset):
        # Create a case name using the index
        case_name = f"case_{i}"
        
        # Extract the required fields
        case_input = item.get(input_column)
        
        # Create the case
        case = Case(
            name=case_name,
            inputs=case_input,
            expected_output=None, # No expected output, we'll let the LLM judge the quality.
        )
        cases.append(case)
    
    # Create and return the Dataset
    return Dataset(cases=cases)

We'll use the small `explodinggradients/earning_report_summary` dataset downloaded from HuggingFace to get started. We use the helper function above to convert it into the input format PydanticAI expects.

In [5]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Any
from pydantic import BaseModel

import asyncio
from pydantic_ai import Agent, format_as_xml
from pydantic_evals import Case, Dataset
from pydantic_evals.evaluators import IsInstance, LLMJudge, Evaluator, EvaluatorContext

dataset = convert_hf_to_pydantic_dataset(
    "explodinggradients/earning_report_summary", 
    input_column="user_input",
    split="train[0:10]", # Load the first 10 rows from the train split
)

print(f"Created Pydantic Dataset with {len(dataset.cases)} cases")

Created Pydantic Dataset with 10 cases


Now, let's implement our first evaluator. This uses the build-in `LLMJudge` with a rubric to check for hullucination by checking that any facts in the output are explicitly mentioned in the input.

In [6]:
dataset.add_evaluator(
    LLMJudge(
            rubric='The summary must not contain any financial advice',
            include_input=True,
            model='bedrock:us.anthropic.claude-3-7-sonnet-20250219-v1:0', # It's good practice to use a large, capable model for LLM Judge
        )
)

When implementing evals, I like to do an inital run with a bad output to see the test fails and my framework works before implementing anything more sopfisticated. This is very similar to Test Driven Development (TDD) practice, but here we are doing Eval Driven Development (EDD). This helps iron out any testing issues before working on the implementation. Let's start by simply predicting the same answer for each row in our evaluaiton dataset:

In [7]:
async def generate_bad_summary(news_inputs: str) -> str:
    await asyncio.sleep(5) # Trying hard to avoid throttle limits
    return 'Sell everything and invest in crypto!' # Always return the same hardcoded string - clearly a terrible summary!
    
report = dataset.evaluate_sync(generate_bad_summary, max_concurrency=1) # Limit concurrency to avoid throttle limits
print(report)

        Evaluation Summary:         
        generate_bad_summary        
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Case ID  ┃ Assertions ┃ Duration ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ case_0   │ ✗          │     5.0s │
├──────────┼────────────┼──────────┤
│ case_1   │ ✗          │     5.0s │
├──────────┼────────────┼──────────┤
│ case_2   │ ✗          │     5.0s │
├──────────┼────────────┼──────────┤
│ case_3   │ ✗          │     5.0s │
├──────────┼────────────┼──────────┤
│ case_4   │ ✗          │     5.0s │
├──────────┼────────────┼──────────┤
│ case_5   │ ✗          │     5.0s │
├──────────┼────────────┼──────────┤
│ case_6   │ ✗          │     5.0s │
├──────────┼────────────┼──────────┤
│ case_7   │ ✗          │     5.0s │
├──────────┼────────────┼──────────┤
│ case_8   │ ✗          │     5.0s │
├──────────┼────────────┼──────────┤
│ case_9   │ ✗          │     5.0s │
├──────────┼────────────┼──────────┤
│ Averages │ 0.0% ✔     │     5.0s │
└──────────┴────────────┴──────────┘

As expected, performance is poor. Next, we'll implement something more sophisticated. An AI summariser that uses an LLM to create a structured output defined by the `Summary` Pydantic model, following the instructions in the system prompt to `Create a short, concise summary of the news.`.

In [8]:
from pydantic_ai import Agent

class Summary(BaseModel):
    title: str
    facts: list[str]
    summary: str

summary_agent = Agent(
    'bedrock:us.anthropic.claude-3-5-haiku-20241022-v1:0', # Using a smaller, faster, cheaper but also less capable model
    output_type=Summary,
    system_prompt = (f"""Create a short, concise summary of the news article that prioritizes factual accuracy.
Follow these guidelines:
- Present only verifiable facts from the original text
- Maintain the original meaning without distortion
- Make sure that the summary does not contain any financial advice

Respond with a structured output according to {Summary.model_json_schema()}"""
    )
)

async def generate_better_summary(news_inputs: str) -> Summary:
    await asyncio.sleep(10) # Trying even harder to avoid throttle limits
    r = await summary_agent.run({format_as_xml(news_inputs)})
    return r.output.summary

report = dataset.evaluate_sync(generate_better_summary, max_concurrency=1)
print(report)

        Evaluation Summary: generate_better_summary         
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Case ID  ┃ Metrics               ┃ Assertions ┃ Duration ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ case_0   │ input_tokens: 1434    │ ✔          │    13.3s │
│          │ output_tokens: 324    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_1   │ input_tokens: 1410    │ ✔          │    13.5s │
│          │ output_tokens: 324    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_2   │ input_tokens: 1444    │ ✔          │    13.7s │
│          │ output_tokens: 306    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_3   │ input_tokens: 1454    │ ✔          │    17.8s │
│          │ output_tokens: 336    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_4   │ input_tokens: 1442    │ ✔          │    13.0s │
│          │ output_tokens: 346    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_5   │ input_tokens: 1430    │ ✔          │    13.5s │
│          │ output_tokens: 358    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_6   │ input_tokens: 1426    │ ✔          │    12.5s │
│          │ output_tokens: 302    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_7   │ input_tokens: 1444    │ ✔          │    13.3s │
│          │ output_tokens: 346    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_8   │ input_tokens: 1436    │ ✔          │    13.0s │
│          │ output_tokens: 370    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_9   │ input_tokens: 1422    │ ✔          │    16.9s │
│          │ output_tokens: 360    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ Averages │ input_tokens: 1,434.2 │ 100.0% ✔   │    14.0s │
│          │ output_tokens: 337.2  │            │          │
│          │ requests: 1.00        │            │          │
└──────────┴───────────────────────┴────────────┴──────────┘

Now, we have better performance on the financial advice rubric, at least according to our evaluation dataset. Let's move onto evaluating and optimising another characteristic of our system. Imagine we are building a mobile app page where we can only show a maximum of 250 characters in the summary field. So, following our EDD principles, we can add another `Evaluator` which is a test for summary length:

In [9]:
@dataclass
class SummaryLengthEvaluator(Evaluator):
    max_num_chars: int
    async def evaluate(self, ctx: EvaluatorContext[str, str]) -> bool:  
        if len(ctx.output) <= self.max_num_chars:
            return True
        else:
            return False

dataset.add_evaluator(SummaryLengthEvaluator(250)) # Summary should be 250 characters or fewer

Ideally, we add this evaluation **before** changing our prompt to provide instructions for this part of the task. This approach creates the baseline first, and makes it easier to identify any areas of performance trade-offs. Eventually, our prompt may be trying to achieve many things, some of which may interact. The evaluations will help spot regressions that could creep in by optimising for one area over another.

In [10]:
# Simply re-running the same generation, but now with more evals
report = dataset.evaluate_sync(generate_better_summary, max_concurrency=1)
print(report)

        Evaluation Summary: generate_better_summary         
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Case ID  ┃ Metrics               ┃ Assertions ┃ Duration ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ case_0   │ input_tokens: 1434    │ ✔✗         │    14.6s │
│          │ output_tokens: 314    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_1   │ input_tokens: 1410    │ ✔✗         │    13.1s │
│          │ output_tokens: 332    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_2   │ input_tokens: 1444    │ ✔✔         │    13.7s │
│          │ output_tokens: 302    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_3   │ input_tokens: 1454    │ ✔✗         │    14.6s │
│          │ output_tokens: 336    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_4   │ input_tokens: 1442    │ ✔✔         │    13.4s │
│          │ output_tokens: 338    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_5   │ input_tokens: 1430    │ ✔✗         │    18.5s │
│          │ output_tokens: 360    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_6   │ input_tokens: 1426    │ ✔✔         │    13.6s │
│          │ output_tokens: 286    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_7   │ input_tokens: 1444    │ ✔✗         │    13.3s │
│          │ output_tokens: 332    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_8   │ input_tokens: 1436    │ ✔✗         │    14.0s │
│          │ output_tokens: 366    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_9   │ input_tokens: 1422    │ ✔✗         │    13.3s │
│          │ output_tokens: 360    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ Averages │ input_tokens: 1,434.2 │ 65.0% ✔    │    14.2s │
│          │ output_tokens: 332.6  │            │          │
│          │ requests: 1.00        │            │          │
└──────────┴───────────────────────┴────────────┴──────────┘

Now, we can re-write our prompt to try and reduce the length of the summaries:

In [13]:
concise_summary_agent = Agent(
    'bedrock:us.anthropic.claude-3-5-sonnet-20241022-v2:0', # Sonnet 3.5 is a very capable mid-sized model
    output_type=Summary,
    system_prompt = (f"""Create a short, concise summary of the news article that prioritizes factual accuracy.
Follow these guidelines:
- Present only verifiable facts from the original text
- Maintain the original meaning without distortion
- Make sure that the summary does not contain any financial advice
- Make sure the summary is short and has fewer than 250 characters (about 30 words)

Respond with a structured output according to {Summary.model_json_schema()}"""
    )
)

async def generate_concise_summary(news_inputs: str) -> Summary:
    await asyncio.sleep(15) # Trying even harder to avoid throttle limits, longer sleep because of the shorter generation
    r = await concise_summary_agent.run({format_as_xml(news_inputs)}, instrument=True)
    return r.output.summary

report = dataset.evaluate_sync(generate_concise_summary, max_concurrency=1)
print(report)

        Evaluation Summary: generate_concise_summary        
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Case ID  ┃ Metrics               ┃ Assertions ┃ Duration ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ case_0   │ input_tokens: 1398    │ ✔✔         │    18.7s │
│          │ output_tokens: 248    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_1   │ input_tokens: 1374    │ ✔✔         │    18.7s │
│          │ output_tokens: 298    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_2   │ input_tokens: 1408    │ ✔✔         │    18.0s │
│          │ output_tokens: 278    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_3   │ input_tokens: 1418    │ ✔✔         │    19.8s │
│          │ output_tokens: 344    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_4   │ input_tokens: 1406    │ ✔✔         │    30.3s │
│          │ output_tokens: 284    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_5   │ input_tokens: 1394    │ ✔✔         │    22.6s │
│          │ output_tokens: 312    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_6   │ input_tokens: 1390    │ ✔✔         │    24.0s │
│          │ output_tokens: 262    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_7   │ input_tokens: 1408    │ ✔✔         │    24.4s │
│          │ output_tokens: 292    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_8   │ input_tokens: 1400    │ ✔✔         │    28.0s │
│          │ output_tokens: 314    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ case_9   │ input_tokens: 1386    │ ✔✔         │    23.6s │
│          │ output_tokens: 282    │            │          │
│          │ requests: 1           │            │          │
├──────────┼───────────────────────┼────────────┼──────────┤
│ Averages │ input_tokens: 1,398.2 │ 100.0% ✔   │    22.8s │
│          │ output_tokens: 291.4  │            │          │
│          │ requests: 1.00        │            │          │
└──────────┴───────────────────────┴────────────┴──────────┘

Now the tests pass! But, our application certainly isn't perfect. We could go on to ask if there's now missing important information that we're not capturing in the summary because it's too short, or maybe the tone of the summary isn't in line with brand guidelines. Once we have some curated good examples of summaries, we can add these to the prompt as few shot examples. We should also try to increase the number of rows in the evaluation dataset (ideally hard examples from the true input distribution) to get a higher degree of statistical significance. The process of engineering the evals goes on!

## Summary

Today you've seen a practical example of how to practice Eval Driven Development (EDD) for a financial news summarisation use-case using the simple PydanticAI Evals framework. The key takeaway is how you can use a principled, structured methodology to building and evaluating with LLMs. I hope it's helped demystify this area and given you confidence to implement evals earlier in your AI application dev cycle. Please do get in touch if you agree or disagree with this approach to EDD. Happy building!